In [1]:
import pandas as pd
import psycopg2

In [2]:
conn = psycopg2.connect(host="winkl-production.cpsgstz5cgos.us-west-2.rds.amazonaws.com",port="5432",database="winkl_production", user="winkl", password="Winkl911911")
cur = conn.cursor()

In [5]:
# Influencers who accepted to campaign
cur.execute('''SELECT id,shortlist_id,campaign_profile_id FROM "shortlisted_users" WHERE (accepted = true)''')
shortlisted_user = pd.DataFrame(cur.fetchall(),columns=["shortlisted_user_id", "shortlist_id", "campaign_profile_id"])
display(shortlisted_user)

,shortlisted_user_id,shortlist_id,campaign_profile_id
0,53,10,4
1,122,28,4
2,56,12,4
3,59,16,4
4,291,73,1312
...,...,...,...
846,2657,456,1331
847,2661,456,10620
848,2662,456,8819
849,2658,456,8899


In [6]:
# Amount paid to influencer
shortlisted_user_details = pd.DataFrame(columns=["shortlisted_user_detail_id", "shortlisted_user_id", "amount"])

for suid in shortlisted_user["shortlisted_user_id"].tolist():
    cur.execute('SELECT id,shortlisted_user_id,amount FROM shortlisted_user_details WHERE shortlisted_user_id ='+str(suid)+' Limit 1')
    temp = pd.DataFrame(cur.fetchall(),columns=["shortlisted_user_detail_id", "shortlisted_user_id", "amount"])
    shortlisted_user_details = shortlisted_user_details.append(temp)
display(shortlisted_user_details)

,shortlisted_user_detail_id,shortlisted_user_id,amount
0,47,53,4000
0,114,122,900
0,50,56,99
0,53,59,5000
0,673,291,1000
...,...,...,...
0,4602,2657,0
0,4598,2661,0
0,4597,2662,0
0,4601,2658,0


In [7]:
# Merge Amount 
influencer_amount = shortlisted_user.merge(shortlisted_user_details, on=["shortlisted_user_id"])
display(influencer_amount)

,shortlisted_user_id,shortlist_id,campaign_profile_id,shortlisted_user_detail_id,amount
0,53,10,4,47,4000
1,122,28,4,114,900
2,56,12,4,50,99
3,59,16,4,53,5000
4,291,73,1312,673,1000
...,...,...,...,...,...
843,2657,456,1331,4602,0
844,2661,456,10620,4598,0
845,2662,456,8819,4597,0
846,2658,456,8899,4601,0


In [8]:
# Campaign Post Deatails | from shortlist_id
shortlist_id = influencer_amount["shortlist_id"].tolist()
cur.execute('SELECT brief_id, shortlist_id FROM campaign_brief_shortlists WHERE shortlist_id IN'+str(tuple(shortlist_id)))
campaign_brief_shortlist = pd.DataFrame(cur.fetchall(),columns=["brief_id", "shortlist_id"])
campaign_brief_shortlist = campaign_brief_shortlist[campaign_brief_shortlist["brief_id"].notna()]
display(campaign_brief_shortlist)
brief_id = campaign_brief_shortlist["brief_id"].tolist()
cur.execute('SELECT id, brief_id, campaign_post_format_id, num_of_posts FROM brief_post_formats WHERE brief_id IN '+str(tuple(brief_id)))
brief_post_format = pd.DataFrame(cur.fetchall(),columns=["brief_post_format_id", "brief_id", "campaign_post_format_id","num_of_posts"])
display(brief_post_format)

# Merge brief_id and campaign_brief
post_format = brief_post_format.merge(campaign_brief_shortlist,on=["brief_id"])
display(post_format)
post_format.to_csv(r'E:\Winkl Mains\Task_2 Price Prediction\DATA\brief_post_format.csv')

,brief_id,shortlist_id
0,75.0,32
1,79.0,37
2,82.0,39
3,94.0,63
4,97.0,66
...,...,...
132,303.0,482
133,303.0,485
134,306.0,488
135,301.0,490


,brief_post_format_id,brief_id,campaign_post_format_id,num_of_posts
0,683,143,1,4.0
1,684,143,2,0.0
2,685,143,3,0.0
3,609,129,8,20.0
4,457,94,1,4.0
...,...,...,...,...
286,3351,306,1,NaN
287,3352,306,2,3.0
288,3353,306,3,1.0
289,3354,306,11,NaN


,brief_post_format_id,brief_id,campaign_post_format_id,num_of_posts,shortlist_id
0,683,143,1,4.0,138
1,684,143,2,0.0,138
2,685,143,3,0.0,138
3,609,129,8,20.0,110
4,609,129,8,20.0,112
...,...,...,...,...,...
521,3351,306,1,NaN,488
522,3352,306,2,3.0,488
523,3353,306,3,1.0,488
524,3354,306,11,NaN,488


In [25]:
# Filter Post_format | Remove NaN or zero
post_format = post_format[post_format["num_of_posts"].notna()]
post_format = post_format[post_format["num_of_posts"] != 0]
display(post_format)
post_format.to_csv(r'E:\Winkl Mains\Task_2 Price Prediction\DATA\brief_post_format.csv')
# print(post_format.shortlist_id.loc[3])

,brief_post_format_id,brief_id,campaign_post_format_id,num_of_posts,shortlist_id
0,683,143,1,4.0,138
3,609,129,8,20.0,110
4,609,129,8,20.0,112
5,457,94,1,4.0,63
8,1748,210,1,1.0,319
...,...,...,...,...,...
514,3311,303,2,4.0,485
515,3312,303,3,1.0,482
516,3312,303,3,1.0,485
522,3352,306,2,3.0,488


110


In [22]:
temp1 = post_format.drop_duplicates(subset = ["shortlist_id"])
display(temp1)
df_size = len(temp1)
unique_shortlist_id  = temp1["shortlist_id"].tolist()
brief_id_col = temp1["brief_id"].tolist()
dummy_col = [0]*df_size
data_dict = {"shortlist_id":unique_shortlist_id,
             "brief_id":brief_id_col,
             "insta_image":dummy_col,
             "insta_story":dummy_col,
             "insta_video":dummy_col,
             "insta_reel":dummy_col,
             "insta_carousel":dummy_col,
             "youtube_video":dummy_col}
filter_post_format = pd.DataFrame(columns = ["shortlist_id","brief_id","insta_image","insta_story","insta_video","insta_reel","insta_carousel","youtube_video"]
                                 ,data=data_dict)
#[1,2,3,11,12,4] campaign_post_format_id
display(filter_post_format)
for i in post_format.index:
    shid = post_format.shortlist_id.loc[i]
    cfid = post_format.campaign_post_format_id.loc[i]
    
    




,brief_post_format_id,brief_id,campaign_post_format_id,num_of_posts,shortlist_id
0,683,143,1,4.0,138
3,609,129,8,20.0,110
4,609,129,8,20.0,112
5,457,94,1,4.0,63
8,1748,210,1,1.0,319
...,...,...,...,...,...
503,3086,295,2,1.0,472
504,3086,295,2,1.0,473
513,3311,303,2,4.0,482
514,3311,303,2,4.0,485


,shortlist_id,brief_id,insta_image,insta_story,insta_video,insta_reel,insta_carousel,youtube_video
0,138,143,0,0,0,0,0,0
1,110,129,0,0,0,0,0,0
2,112,129,0,0,0,0,0,0
3,63,94,0,0,0,0,0,0
4,319,210,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
121,472,295,0,0,0,0,0,0
122,473,295,0,0,0,0,0,0
123,482,303,0,0,0,0,0,0
124,485,303,0,0,0,0,0,0


0
3
4
5
8
9
13
14
15
16
21
25
26
27
28
30
33
36
39
41
68
71
74
77
80
81
82
83
84
89
90
91
92
93
94
95
96
97
98
108
109
110
111
112
113
114
115
116
117
118
119
139
140
141
142
143
144
145
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
185
186
187
188
194
195
198
199
200
204
205
206
207
208
209
210
211
212
215
216
217
218
219
220
226
227
228
229
230
231
232
233
234
235
246
247
251
252
256
257
258
259
260
261
262
263
264
271
272
278
279
284
285
286
287
291
292
297
298
299
302
303
304
306
307
308
309
314
315
316
317
327
328
329
331
332
336
337
341
342
346
347
348
349
351
353
357
358
360
361
362
363
364
365
366
367
368
381
382
386
387
394
395
396
397
398
399
407
408
411
412
416
417
418
419
420
421
428
429
430
431
436
437
439
440
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
459
464
465
469
470
479
480
481
482
483
484
488
489
493
494
496
497
498
500
503
504
507
508
513
514
515
516
522
523


In [7]:
shortlist_ids = shortlisted_user["shortlist_id"].tolist()
cur.execute('SELECT id,brief_id FROM "campaign_brief_shortlist" WHERE id IN'+str(tuple(shortlist_ids)))
campaign_brief_shortlist = pd.DataFrame(cur.fetchall(),columns=["campaign_brief_shortlist_id", "brief_id"])
display(campaign_brief_shortlist)

,campaign_brief_shortlist_id,brief_id
0,31,74.0
1,32,75.0
2,39,82.0
3,66,97.0
4,73,97.0
5,78,97.0
6,113,129.0
7,115,131.0
8,116,132.0
9,117,97.0


In [3]:
cur.execute(''' SELECT "shortlisted_user_details"."shortlisted_user_id", "shortlisted_user_details"."amount" FROM "shortlisted_user_details" WHERE "shortlisted_user_details"."shortlisted_user_id" IN (SELECT "shortlisted_users"."id" FROM "shortlisted_users" WHERE (paid = true))''')
profileamount = pd.DataFrame(cur.fetchall(),columns=["shortlisted_user_id", "amount"])
display(profileamount)

,shortlisted_user_id,amount
0,53,5000
1,53,4000
2,53,4000
3,53,4000
4,54,1999
...,...,...
732,2351,4000
733,2349,7000
734,2192,13000
735,2346,15000


In [5]:
# Constant IDs
cur.execute('''SELECT "campaign_post_formats"."id", "campaign_post_formats"."platform_id", "campaign_post_formats"."format" FROM "campaign_post_formats"''')
post_format = pd.DataFrame(cur.fetchall(),columns=["campaign_post_format_id", "platform_id", "format"])
display(post_format)
cur.execute('''SELECT "platforms"."id", "platforms"."platform" FROM "platforms"''')
platform = pd.DataFrame(cur.fetchall(),columns=["platform_id", "platforms"])
display(platform)

,campaign_post_format_id,platform_id,format
0,1,1,Image
1,2,1,Story
2,3,1,Video
3,4,2,Video
4,5,3,Video
5,6,4,Image
6,7,4,Video
7,8,5,Video
8,10,4,Idea List Link
9,11,1,Reels


,platform_id,platforms
0,1,Instagram
1,2,YouTube
2,3,TikTok
3,4,Amazon
4,5,Trell


In [26]:
cur.execute(''' SELECT "campaign_profile_post_formats"."campaign_profile_id", "campaign_profile_post_formats"."campaign_post_format_id", "campaign_profile_post_formats"."price" FROM "campaign_profile_post_formats" WHERE (price >1)''')
profileamount = pd.DataFrame(cur.fetchall(),columns=["campaign_profile_id", "campaign_post_format_id", "price"])
cpid = profileamount["campaign_profile_id"].tolist()
display(profileamount)

,campaign_profile_id,campaign_post_format_id,price
0,3,1,355.0
1,3,2,553.0
2,3,3,466.0
3,3,5,654.0
4,3,4,100.0
...,...,...,...
8186,650,4,5000.0
8187,3220,1,100.0
8188,3220,2,10.0
8189,3220,3,100.0


In [9]:
#Youtube Profile
youtubeprofile = profileamount[profileamount["campaign_post_format_id"] == 4]
display(youtubeprofile)

,campaign_profile_id,campaign_post_format_id,price
4,3,4,100.0
18,4,4,34.0
39,11704,4,100000.0
81,10866,4,500.0
85,11967,4,7500.0
...,...,...,...
8058,13198,4,1000.0
8063,10869,4,5000.0
8094,944,4,1500.0
8103,3805,4,12000.0


In [25]:
#Insta Profiles only
# instaimage = profileamount[profileamount["campaign_post_format_id"] == 1]
# iiid = instaimage["campaign_profile_id"].tolist()
cur.execute('SELECT  id,insta_user_id,youtube_channel_id FROM campaign_profiles WHERE id IN'+str(tuple(cpid)))
insta_ids = pd.DataFrame(cur.fetchall(),columns=["campaign_profile_id", "insta_user_id","youtube_channel_id"])
display(insta_ids)

,campaign_profile_id,insta_user_id,youtube_channel_id
0,5,218958.0,NaN
1,22,NaN,3303.0
2,25,NaN,2965.0
3,26,260248.0,NaN
4,29,4383.0,NaN
...,...,...,...
1631,13300,318712.0,NaN
1632,13302,318658.0,NaN
1633,13305,318696.0,NaN
1634,13306,88456.0,NaN


In [38]:
# Clear NaN | Get insta infos
OnlyInsta = insta_ids[insta_ids["insta_user_id"].notna()]
cur.execute('SELECT insta_user_id,"instagram_accounts"."followers", "instagram_accounts"."avg_engagement", "instagram_accounts"."avg_likes", "instagram_accounts"."avg_comments", "instagram_accounts"."gender", "instagram_accounts"."num_of_posts", "instagram_accounts"."total_likes", "instagram_accounts"."total_comments", "instagram_accounts"."follower_increase" FROM "instagram_accounts" WHERE insta_user_id IN'+str(tuple(OnlyInsta["insta_user_id"].tolist())))
InstaInfo = pd.DataFrame(cur.fetchall(),columns=["insta_user_id", "followers","avg_engagement","avg_likes","avg_comments","gender","num_of_posts","total_likes","total_comments","follower_increase"])
display(newarr)
display(InstaInfo)

,campaign_profile_id,insta_user_id,youtube_channel_id
0,5,218958.0,NaN
3,26,260248.0,NaN
4,29,4383.0,NaN
5,33,21131.0,NaN
6,52,128776.0,NaN
...,...,...,...
1631,13300,318712.0,NaN
1632,13302,318658.0,NaN
1633,13305,318696.0,NaN
1634,13306,88456.0,NaN


,insta_user_id,followers,avg_engagement,avg_likes,avg_comments,gender,num_of_posts,total_likes,total_comments,follower_increase
0,71608,17229,3.96,635.17,46.75,None,355.0,9147,939,-0.004277
1,314243,1402,5.69,59.50,20.25,None,89.0,736,258,0.007908
2,9528,27050,1.14,285.92,22.33,Female,672.0,7513,1356,0.009404
3,288339,16113,5.53,752.58,138.42,None,181.0,9031,1661,0.022593
4,269833,58871,0.90,434.33,94.67,Female,1532.0,7358,1136,0.001838
...,...,...,...,...,...,...,...,...,...,...
1586,31568,472799,3.70,17342.58,146.92,Female,1751.0,217193,5083,-0.000004
1587,299008,4340,3.16,135.83,1.33,None,25.0,1682,17,-0.000230
1588,104059,3488,1.67,52.75,5.58,Male,108.0,633,67,0.000000
1589,298222,10286,7.36,659.58,98.00,None,69.0,10605,1443,0.000000


In [47]:
fullInstaInfo = InstaInfo.merge(newarr, on =["insta_user_id"])
display(fullInstaInfo)
FullInstaAmount = profileamount.merge(fullInstaInfo, on=["campaign_profile_id"])
display(FullInstaAmount)
FullInstaAmount.to_csv('E:\Winkl Mains\Task_2 Price Prediction\DATA\InstaProfilePayments.csv')

,insta_user_id,followers,avg_engagement,avg_likes,avg_comments,gender,num_of_posts,total_likes,total_comments,follower_increase,campaign_profile_id,youtube_channel_id
0,71608,17229,3.96,635.17,46.75,None,355.0,9147,939,-0.004277,8453,NaN
1,314243,1402,5.69,59.50,20.25,None,89.0,736,258,0.007908,13033,NaN
2,9528,27050,1.14,285.92,22.33,Female,672.0,7513,1356,0.009404,732,NaN
3,288339,16113,5.53,752.58,138.42,None,181.0,9031,1661,0.022593,10587,NaN
4,269833,58871,0.90,434.33,94.67,Female,1532.0,7358,1136,0.001838,1087,3128.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1586,31568,472799,3.70,17342.58,146.92,Female,1751.0,217193,5083,-0.000004,4804,4436.0
1587,299008,4340,3.16,135.83,1.33,None,25.0,1682,17,-0.000230,11555,NaN
1588,104059,3488,1.67,52.75,5.58,Male,108.0,633,67,0.000000,6877,NaN
1589,298222,10286,7.36,659.58,98.00,None,69.0,10605,1443,0.000000,7407,NaN


,campaign_profile_id,campaign_post_format_id,price,insta_user_id,followers,avg_engagement,avg_likes,avg_comments,gender,num_of_posts,total_likes,total_comments,follower_increase,youtube_channel_id
0,7447,3,1500.0,270387,20168,3.89,753.33,30.50,Female,1025.0,14650,657,0.030294,NaN
1,7447,11,1000.0,270387,20168,3.89,753.33,30.50,Female,1025.0,14650,657,0.030294,NaN
2,7447,12,1500.0,270387,20168,3.89,753.33,30.50,Female,1025.0,14650,657,0.030294,NaN
3,7447,1,1500.0,270387,20168,3.89,753.33,30.50,Female,1025.0,14650,657,0.030294,NaN
4,7447,2,500.0,270387,20168,3.89,753.33,30.50,Female,1025.0,14650,657,0.030294,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7903,69,2,1200.0,83875,17959,6.24,1051.83,68.25,Male,500.0,13837,2218,0.000000,2945.0
7904,69,3,3000.0,83875,17959,6.24,1051.83,68.25,Male,500.0,13837,2218,0.000000,2945.0
7905,69,11,2500.0,83875,17959,6.24,1051.83,68.25,Male,500.0,13837,2218,0.000000,2945.0
7906,69,12,3000.0,83875,17959,6.24,1051.83,68.25,Male,500.0,13837,2218,0.000000,2945.0
